<a href="https://colab.research.google.com/github/shrushti-sd/Prodigy_ML_01/blob/main/Prodigy_ML_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np


In [11]:
df = pd.read_csv('/content/indian_food.csv')

In [12]:
def estimate_calories(food_name, serving_size=1):
    food_info = df[df['name'] == food_name].iloc[0]

    calorie_map = {
        'dessert': 350,
        'main course': 400,
        'snack': 200,
        'starter': 150
    }
    base_calories = calorie_map.get(food_info['course'], 300)

    ingredients = food_info['ingredients'].lower()
    if 'ghee' in ingredients or 'butter' in ingredients:
        base_calories += 80
    if 'sugar' in ingredients or 'jaggery' in ingredients:
        base_calories += 50
    if 'rice' in ingredients or 'wheat' in ingredients:
        base_calories += 70
    if 'milk' in ingredients or 'cream' in ingredients:
        base_calories += 60
    return round(base_calories * serving_size, 1)

In [13]:
class FoodRecognizer:
    def __init__(self):
        self.label_encoder = LabelEncoder()
        self.label_encoder.fit(df['name'])
        self.num_classes = len(self.label_encoder.classes_)

        self.model = self._create_dummy_model()

    def _create_dummy_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(224, 224, 3)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(self.num_classes, activation='softmax')
        ])
        return model

    def recognize_food(self, img_path):
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0

        predictions = self.model.predict(img_array)
        predicted_class = np.argmax(predictions[0])
        food_name = self.label_encoder.inverse_transform([predicted_class])[0]

        return food_name

In [14]:
class FoodTracker:
    def __init__(self):
        self.recognizer = FoodRecognizer()

    def analyze_meal(self, image_path, serving_size=1):
        try:
            food_name = self.recognizer.recognize_food(image_path)

            calories = estimate_calories(food_name, serving_size)

            food_info = df[df['name'] == food_name].iloc[0]

            return {
                'food_name': food_name,
                'calories': calories,
                'serving_size': serving_size,
                'diet_type': food_info['diet'],
                'region': food_info['region'],
                'course': food_info['course'],
                'ingredients': food_info['ingredients']
            }

        except Exception as e:
            return {'error': str(e)}

In [15]:
if __name__ == "__main__":
    tracker = FoodTracker()


    sample_food = df.sample(1).iloc[0]
    print(f"Analyzing: {sample_food['name']}")

    result = tracker.analyze_meal(None, serving_size=1.5)

    print("\nAnalysis Results:")
    if 'error' in result:
        print(f"Error analyzing meal: {result['error']}")
    else:
        print(f"Food: {result['food_name']}")
        print(f"Calories: {result['calories']} kcal (for {result['serving_size']} serving)")
        print(f"Diet: {result['diet_type']}")
        print(f"Region: {result['region']}")
        print(f"Course: {result['course']}")
        print(f"Main Ingredients: {result['ingredients']}")

Analyzing: Kolim Jawla

Analysis Results:
Error analyzing meal: path should be path-like or io.BytesIO, not <class 'NoneType'>


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
